In [1]:
import pandas as pd
import geopandas as gpd

import numpy as np
import matplotlib.pyplot as plt

from shapely import wkt

import re

In [2]:
colnames = ['a', 'b', 'c', 'd', 'e', 'f',
       'g', 'h', 'i', 'j',
       'k', 
        'AD', 'ED', 'County', 'EDAD Status', 'Event', 'Party/Independent Body',
       'Office/Position Title', 'District Key', 'VoteFor', 'Unit Name',
       'Tally']

good_cols = ['AD', 'ED', 'County', 'EDAD Status', 'Event', 'Party/Independent Body',
       'Office/Position Title', 'District Key', 'VoteFor', 'Unit Name',
       'Tally']

In [3]:
def import_clean(filepath,good_cols,colnames):
    df = pd.read_csv(filepath,header=None, names=colnames)
    df = df[good_cols]
    df['precinct'] = 'AD: ' + df['AD'].astype(str) + ' ' + 'ED: ' + df['ED'].astype(str).str.zfill(3)
    df['ed_name'] = df['AD'].astype(str) + df['ED'].astype(str).str.zfill(3)
    return df

def rearrange(df,candidates):
    df = df[df['Unit Name'].isin(candidates)]
    df = df[['precinct','ed_name','Unit Name','Tally']]
    df.columns = ['precinct','ed_name','candidate','votes']
    df = df.groupby(['ed_name','candidate'])['votes'].sum().reset_index()
    df['ed_total'] = df.groupby('ed_name')['votes'].transform('sum')
    df['vote_share'] = round(df['votes'] / df['ed_total'] * 100,2)
    df['vote_share'] = pd.to_numeric(df['vote_share'], errors='coerce')
    return df

In [120]:
p16 = pd.read_csv('../data/pres_2016.csv') 
p20 = import_clean('../data/pres_2020.csv',good_cols,colnames)
p24 = import_clean('../data/pres_2024.csv',good_cols,colnames)

primary16 = pd.read_csv('../data/dem_pres_primary_2016.csv') 
primary20 = import_clean('../data/dem_pres_primary_2020.csv',good_cols,colnames)

C:\Users\samtg\AppData\Local\Temp\ipykernel_16460\907058922.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath,header=None, names=colnames)


In [15]:
def ed_maker(df):
    df['precinct'] = 'AD: ' + df['AD'].astype(str) + ' ' + 'ED: ' + df['ED'].astype(str).str.zfill(3)
    df['ed_name'] = df['AD'].astype(str) + df['ED'].astype(str).str.zfill(3)
    return df

In [121]:
p16 = ed_maker(p16)

In [5]:
def rearrange_combined(
    df,
    canonical_map=None,
    unit_col='Unit Name',
    precinct_col='precinct',
    ed_col='ed_name',
    tally_col='Tally'
):
    """
    Collapse multiple ballot lines into canonical candidate names and compute vote shares.

    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe containing at least columns named by unit_col and tally_col.
    canonical_map : dict
        Mapping of keyword -> canonical name. Keys will be matched case-insensitively
        inside the unit_col text. Example: {'trump': 'Trump', 'clinton': 'Clinton'}
        If None, defaults to {'trump': 'Trump', 'clinton': 'Clinton'}.
    unit_col / precinct_col / ed_col / tally_col : str
        Column names in df (defaults chosen to match your example).
    """
    if canonical_map is None:
        canonical_map = {'trump': 'Trump', 'clinton': 'Clinton'}


    df = df.copy()


    df[tally_col] = pd.to_numeric(df[tally_col], errors='coerce').fillna(0)


    lower_unit = df[unit_col].astype(str).str.lower()

    df['candidate_std'] = np.nan

    for keyword, canonical in canonical_map.items():
        if not isinstance(keyword, str):
            continue
        mask = lower_unit.str.contains(re.escape(keyword.lower()), na=False)
        df.loc[mask, 'candidate_std'] = canonical




    df = df[df['candidate_std'].notna()].copy()


    df = df[[precinct_col, ed_col, 'candidate_std', tally_col]]
    df.columns = ['precinct', 'ed_name', 'candidate', 'votes']


    df = df.groupby(['ed_name', 'candidate'])['votes'].sum().reset_index()


    df['ed_total'] = df.groupby('ed_name')['votes'].transform('sum')
    df['vote_share'] = round(df['votes'] / df['ed_total'] * 100, 2)
    df['vote_share'] = pd.to_numeric(df['vote_share'], errors='coerce')

    return df

In [122]:
p16 = p16[p16['County'] == 'Bronx']
p20 = p20[p20['County'] == 'Bronx']
p24 = p24[p24['County'] == 'Bronx']

primary16 = primary16[primary16['County'] == 'Bronx']
primary20 = primary20[primary20['County'] == 'Bronx']

In [123]:
canonical_map = {
    'trump': 'Trump',
    'donald j. trump': 'Trump',      
    'clinton': 'Clinton',
    'hillary': 'Clinton'            
}

p16 = rearrange_combined(p16, canonical_map=canonical_map)

C:\Users\samtg\AppData\Local\Temp\ipykernel_16460\169926661.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trump' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'candidate_std'] = canonical


In [124]:
canonical_map = {
    'trump': 'Trump',
    'donald j. trump': 'Trump',      
    'biden': 'Biden',
    'joseph': 'Biden'            
}

p20 = rearrange_combined(p20, canonical_map=canonical_map)

C:\Users\samtg\AppData\Local\Temp\ipykernel_16460\169926661.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trump' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'candidate_std'] = canonical


In [20]:
p24['Unit Name'].unique()

array(['Public Counter', 'Manually Counted Emergency',
       'Absentee / Military', 'Federal', 'Affidavit',
       'Kamala D. Harris / Tim Walz (Democratic)',
       'Donald J. Trump / JD Vance (Republican)',
       'Donald J. Trump / JD Vance (Conservative)',
       'Kamala D. Harris / Tim Walz (Working Families)', 'Scattered'],
      dtype=object)

In [125]:
canonical_map = {
    'trump': 'Trump',
    'donald j. trump': 'Trump',      
    'harris': 'Harris',
    'kamala': 'Harris'            
}

p24 = rearrange_combined(p24, canonical_map=canonical_map)

C:\Users\samtg\AppData\Local\Temp\ipykernel_16460\169926661.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trump' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'candidate_std'] = canonical


In [126]:
p16 = p16[['ed_name','candidate','vote_share']]
p16.columns = [['ed_name','candidate','vs16']]

p20 = p20[['ed_name','candidate','vote_share']]
p20.columns = [['ed_name','candidate','vs20']]

p24 = p24[['ed_name','candidate','vote_share']]
p24.columns = [['ed_name','candidate','vs24']]

In [27]:
primary16['Unit Name'].unique()

array(['Public Counter', 'Emergency', 'Absentee/Military', 'Federal',
       'Affidavit', 'Bernie Sanders', 'Hillary Clinton'], dtype=object)

In [127]:
primary16 = ed_maker(primary16)

In [128]:
canonical_map = {
    'bernie': 'Sanders',
    'hillary': 'Clinton',             
}

primary16 = rearrange_combined(primary16, canonical_map=canonical_map)

C:\Users\samtg\AppData\Local\Temp\ipykernel_16460\169926661.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Sanders' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'candidate_std'] = canonical


In [32]:
primary20['Unit Name'].unique()

array(['Public Counter', 'Manually Counted Emergency',
       'Absentee / Military', 'Federal', 'Affidavit', 'Pete Buttigieg',
       'Amy Klobuchar', 'Joseph R. Biden', 'Tulsi Gabbard',
       'Bernie Sanders', 'Michael R. Bloomberg', 'Tom Steyer',
       'Michael Bennet', 'Elizabeth Warren', 'Andrew Yang',
       'Deval Patrick'], dtype=object)

In [129]:
canonical_map = {
    'bernie': 'Sanders',
    'biden': 'Biden',             
}

primary20 = rearrange_combined(primary20, canonical_map=canonical_map)

C:\Users\samtg\AppData\Local\Temp\ipykernel_16460\169926661.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Sanders' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'candidate_std'] = canonical


In [130]:
p16.columns = [col[0] if isinstance(col, tuple) else col for col in p16.columns]
p16_t = p16[p16['candidate'] == 'Trump']
p20.columns = [col[0] if isinstance(col, tuple) else col for col in p20.columns]
p20_t = p20[p20['candidate'] == 'Trump']
p24.columns = [col[0] if isinstance(col, tuple) else col for col in p24.columns]
p24_t = p24[p24['candidate'] == 'Trump']

In [132]:
p16_t = p16_t.merge(p20_t, on = ['ed_name','candidate'])
p16_t = p16_t.merge(p24_t, on = ['ed_name','candidate'])

In [136]:
p16_t.shape

(659, 5)

In [ ]:
p16_t[p16_t['vs20'] > p16_t['vs16']].shape

(41, 5)

In [135]:
p16_t[(p16_t['vs20'] > p16_t['vs16']) & ((p16_t['vs20'] > p16_t['vs24']))].shape

(53, 5)

In [152]:
p16_t['first_diff'] = p16_t['vs20'] - p16_t['vs16']
p16_t['second_diff'] = p16_t['vs24'] - p16_t['vs20']
p16_t['third_diff'] = p16_t['vs24'] - p16_t['vs16']

In [144]:
primary16s = primary16[primary16['candidate'] == 'Sanders']

In [146]:
primary20s = primary20[primary20['candidate'] == 'Sanders']

In [147]:
primary16s = primary16s[['ed_name','vote_share']]
primary16s.columns = ['ed_name','san16vs']
primary20s = primary20s[['ed_name','vote_share']]
primary20s.columns = ['ed_name','san20vs']

In [148]:
sanders = primary16s.merge(primary20s,on='ed_name')

In [153]:
bronx_cw = p16_t.merge(sanders,on='ed_name')

In [159]:
bronx_cw = bronx_cw[['ed_name', 'vs16', 'vs20', 'vs24', 'first_diff',
       'second_diff', 'third_diff', 'san16vs', 'san20vs']]

In [161]:
bronx_cw.to_csv('../data/bronx_pres_primary.csv',index=False)